In [1]:
#export
############################################################################
### MOST OF THE CODE BELOW WAS TAKEN FROM THE ORIGINAL PYTORCH REPO ###  
### https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py ###
############################################################################

import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from collections import OrderedDict

In [2]:
#export
model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
}

In [3]:
#export
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

In [4]:
#export
class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

In [5]:
#export
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))

In [6]:
#export
class _SEBlock(nn.Module):
    def __init__(self, in_ch, r=16):
        super(_SEBlock, self).__init__()
        self.linear_1 = nn.Linear(in_ch, in_ch//r)
        self.linear_2 = nn.Linear(in_ch//r, in_ch)

    def forward(self,x):
        input_x = x # shape of input_x : (c,w,h)
        x = x.view(*(x.shape[:-2]),-1).mean(-1) # shape of x : (c,)
        x = F.relu(self.linear_1(x), inplace=True) # # shape of x : (c/r,)
        x = self.linear_2(x) # shape of x : (c,)
        x = x.unsqueeze(-1).unsqueeze(-1) # shape of x : (c,1,1)
        x = torch.sigmoid(x) # shape of x : (c,1,1)

        x = torch.mul(input_x, x) # After element wise mult ,shape of x : (c,w,h)
        return x 

In [7]:
#export
class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=2,num_blocks=4):

        super(DenseNet, self).__init__()
        self.num_blocks = num_blocks
        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))


        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2
                seBlock = _SEBlock(in_ch=num_features)
                self.features.add_module('sEBlock%d' % (i + 1), seBlock)

        self.num_features = num_features
        
        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=7*(2**(4-self.num_blocks)), stride=1).view(features.size(0), -1)
        out = self.classifier(out)
        return out

In [8]:
#export
def densenet121(pretrained=False, num_layers = 0, drop_rate = 0,num_classes=8, **kwargs):
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    if num_layers > 42:
        block_config = (6, 12, 24, num_layers - 42)
        transition = 3
        num_blocks = 4
    elif num_layers > 18:
        block_config = (6, 12, num_layers - 18)
        transition = 2
        num_blocks = 3
    elif num_layers > 6:
        block_config = (6, num_layers - 6)
        transition = 1
        num_blocks = 2
    elif num_layers > 0:
        block_config = (num_layers)
        num_blocks = 1


    model = DenseNet(num_init_features=64, growth_rate=32, block_config=block_config, drop_rate=drop_rate,num_blocks=num_blocks,
                     **kwargs)
    if pretrained:
        pattern = re.compile(r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')
        origin_model = model_zoo.load_url(model_urls['densenet121'])

        for key in list(origin_model.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                origin_model[new_key[9:]] = origin_model[key]
                del origin_model[key]

        model_dict = model.state_dict()
        # Select only required keys
        origin_model = {k: v for k, v in origin_model.items() if k in model_dict}
        # Existing state dict is overwritten 
        model_dict.update(origin_model)
        # load the updated state dict
        model.load_state_dict(model_dict)
        
        # Final batch norm
        model.features.add_module('norm5', nn.BatchNorm2d(model.num_features))

        # Linear layer
        model.classifier = nn.Linear(model.num_features, num_classes)
        
        
    return model

In [9]:
densenet121(pretrained=True,num_layers=47)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [13]:
!python notebook2script.py DenseNet.ipynb

Converted DenseNet.ipynb to src\DenseNet.py


In [2]:
torch.save??

In [3]:
def a(x,y):
    return x,y

In [5]:
y = [a(x,y)[0] for x,y in zip(range(4),range(1,5))]

In [6]:
y

[0, 1, 2, 3]